In [33]:
# import packages
import pandas as pd
# import seaborn as sb
import matplotlib.pyplot as plt
import numpy as np
import datetime
import re
from sklearn.externals import joblib 
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from scipy.interpolate import interpn
from collections import defaultdict

In [34]:
# suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [35]:
sales_means =pd.read_csv('./data/sales_means.csv', sep='|').drop(columns=["predicted_promotion"])
scaler = joblib.load('scaler.pkl') 
df_test = pd.read_csv('data/orders0206_test.csv', sep='|', parse_dates=['time'])
df_test['date'] = [d.date() for d in df_test['time']]
df_train = pd.read_csv('data/orders0206_train.csv', sep='|', parse_dates=['time'])
df_train['date'] = [d.date() for d in df_train['time']]

In [36]:
X_train = sales_means[sales_means['weekGroup']<=9]
X_cv =  sales_means[sales_means['weekGroup']==10]
X_test = sales_means[sales_means['weekGroup']==11]
Y_train = X_train['count']
Y_cv = X_cv['count']
Y_test = X_test['count']
del X_train['count']
del X_cv['count']
del X_test['count']
del X_train['weekGroup']
del X_cv['weekGroup']
del X_test['weekGroup']

In [37]:
del X_train['itemID']
del X_cv['itemID']

In [38]:
from xgboost import XGBRegressor
model = XGBRegressor(
    booster = 'gbtree', 
    colsample_bytree = 0.8, 
    gamma = 0, 
    learning_rate = 0.1, 
    max_depth= 3, 
    min_child_weight= 1, 
    n_estimators= 300, 
    objective= 'reg:logistic', 
    seed= 20, 
    silent= 1, 
    subsample= 0.8)
model.fit(
    X_train, 
    Y_train, 
    eval_metric="rmse", 
    eval_set=[(X_train, Y_train), (X_cv, Y_cv)], 
    verbose=True, 
    early_stopping_rounds = 10)

[0]	validation_0-rmse:0.44137	validation_1-rmse:0.44287
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 10 rounds.
[1]	validation_0-rmse:0.39802	validation_1-rmse:0.39959
[2]	validation_0-rmse:0.35920	validation_1-rmse:0.36074
[3]	validation_0-rmse:0.32439	validation_1-rmse:0.32598
[4]	validation_0-rmse:0.29314	validation_1-rmse:0.29485
[5]	validation_0-rmse:0.26507	validation_1-rmse:0.26681
[6]	validation_0-rmse:0.23980	validation_1-rmse:0.24166
[7]	validation_0-rmse:0.21707	validation_1-rmse:0.21899
[8]	validation_0-rmse:0.19661	validation_1-rmse:0.19857
[9]	validation_0-rmse:0.17820	validation_1-rmse:0.18020
[10]	validation_0-rmse:0.16164	validation_1-rmse:0.16358
[11]	validation_0-rmse:0.14676	validation_1-rmse:0.14870
[12]	validation_0-rmse:0.13340	validation_1-rmse:0.13528
[13]	validation_0-rmse:0.12139	validation_1-rmse:0.12313
[14]	validation_0-rmse:0.11060	validation_1-rmse:0.11

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints=None,
             learning_rate=0.1, max_delta_step=0, max_depth=3,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             n_estimators=300, n_jobs=0, num_parallel_tree=1,
             objective='reg:logistic', random_state=20, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, seed=20, silent=1, subsample=0.8,
             tree_method=None, validate_parameters=False, verbosity=None)

In [39]:
import plotly_express as px
columns = X_train.columns
feature_importances = pd.DataFrame({'columns': columns,'importance':model.feature_importances_})
feature_importances = feature_importances.sort_values(by='importance',ascending=False)
px.bar(feature_importances,x='columns',y='importance')

In [40]:
def evaluate_result(y: dict, y_pred: dict):
    monetary_value = 0
    y_pred = defaultdict(int, y_pred)  # return prediction of 0 for items without prediction
    
    for item in set(y_pred).difference(set(y)):
        y[item] = 0  # make sure that all items for which a demand has been predicted are contained in the actual demands
    
    for item, demand in y.items():
        predicted_demand = y_pred[item]
        price = product_prices[item]
        monetary_value += price * min(demand, predicted_demand)
        if predicted_demand > demand:
            monetary_value -= .6 * price * (predicted_demand - demand)
            
    return monetary_value

In [41]:
df_info = pd.read_csv('data/infos.csv', sep='|', index_col='itemID')
df_items = pd.read_csv('data/items.csv', sep='|', index_col='itemID')
df_items.head()

,brand,manufacturer,customerRating,category1,category2,category3,recommendedRetailPrice
itemID,,,,,,,
1,0,1,4.38,1,1,1,8.84
2,0,2,3.00,1,2,1,16.92
3,0,3,5.00,1,3,1,15.89
4,0,2,4.44,1,2,1,40.17
5,0,2,2.33,1,1,1,17.04


In [42]:
# actual demand
y = df_test.groupby(by='itemID')['order'].sum().to_dict()

# baseline 1 (average demand of previous 14 days)
y_baseline1 = df_train[df_train['time'] >= '2018-05-19'].groupby(by='itemID')['order'].sum().to_dict()

# baseline 2 (average demand of previous half year)
total_orders = df_train.groupby(by='itemID')['order'].sum().to_dict()
total_observed_days = (df_train['time'].dt.normalize().max() - df_train['time'].dt.normalize().min()).days
y_baseline2 = {item: orders / total_observed_days * 14 for item, orders in total_orders.items()}  # 14-day avg. demand

df_info = pd.read_csv('data/infos.csv', sep='|', index_col='itemID')
df_items = pd.read_csv('data/items.csv', sep='|', index_col='itemID')
product_prices = df_info['simulationPrice'].to_dict()

In [ ]:
# apply to unseen data
y_xgboost_all = dict()
for prod in X_test.itemID.unique():
    train_predict = pd.DataFrame(scaler.inverse_transform(pd.DataFrame(model.predict(X_test[X_test.itemID == prod].drop(['itemID'], axis=1))))).rename(columns={0:'predicted_count'})
    # train_predict["actual_count"] = test[test.itemID == prod]["order"].sum()
    y_xgboost_all[prod] = int(train_predict["predicted_count"].sum().round())
#     if prod in df_train.itemID.unique():
#         y_xgboost_all[prod] = int(train_predict["predicted_count"].sum().round())
#     else:
#         y_xgboost_all[prod] = 0

In [ ]:
# perfect result
print(f'Perfect Result: {evaluate_result(y, y):.2f}')

# baseline 1
print(f'Baseline 1: {evaluate_result(y, y_baseline1):.2f}')

# baseline 2
print(f'Baseline 2: {evaluate_result(y, y_baseline2):.2f}')

# random forest
print(f'XGBoost: {evaluate_result(y, y_xgboost_all):.2f}')
# 953796.09